# main

To run the neural noise analysis, first modify the parameters in the following files:
- `src/rs/full/preprocessing/evt_preprocessing.py`
- `src/rs/full/analysis/spectral_slopes.py`

In [2]:
import os
import sys

In [2]:
import evt_preprocessing

In [ ]:
import spectral_slopes